In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import torch
from tqdm import tqdm

In [111]:
dat = pd.read_csv('merged_data.csv')
dat = dat.drop(columns=['kWh'])
dat = dat.rename(columns={'Unnamed: 0': 'datetime'})
dat['datetime'] = pd.to_datetime(dat['datetime'])
dat.head()

,datetime,MWh,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW],temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH
0,2019-01-01 00:00:00,139.525004,0.0,0.0,21344.8514,4302.6977,4.106700,5.972900,7.426800,4.028100,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
1,2019-01-01 01:00:00,129.716036,0.0,0.0,23052.3310,4596.5916,4.106700,5.972900,7.426800,4.028100,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
2,2019-01-01 02:00:00,133.398074,0.0,0.0,24969.9701,4478.5564,3.195182,5.538617,7.017926,3.167118,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
3,2019-01-01 03:00:00,135.133852,0.0,0.0,27082.9626,4323.3712,2.491792,5.249173,6.700315,2.563862,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
4,2019-01-01 04:00:00,131.699424,0.0,0.0,26890.9717,4231.8283,1.996528,5.104570,6.473967,2.218332,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0


In [112]:
dat.shape

(26304, 18)

In [113]:
# Extract day of the week (0 = Monday, 6 = Sunday)
dat['weekday'] = dat['datetime'].dt.weekday
dat['month'] = dat['datetime'].dt.month
dat['time'] = dat['datetime'].dt.time

# Encode 'month' and 'weekday' as categorical variables
dat['month'] = dat['month'].astype('category')
dat['weekday'] = dat['weekday'].astype('category')

In [114]:
# Perform one-hot encoding on 'month' and 'weekday'
df_encoded = pd.get_dummies(dat, columns=['month', 'weekday', "time"], prefix=['month', 'weekday', 'time'])

# remove datetime
df_encoded = df_encoded.drop('datetime', axis=1)

In [115]:
df_encoded

,MWh,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW],temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,CH_AT,...,time_14:00:00,time_15:00:00,time_16:00:00,time_17:00:00,time_18:00:00,time_19:00:00,time_20:00:00,time_21:00:00,time_22:00:00,time_23:00:00
0,139.525004,0.0,0.0,21344.8514,4302.6977,4.106700,5.972900,7.426800,4.028100,700.0,...,False,False,False,False,False,False,False,False,False,False
1,129.716036,0.0,0.0,23052.3310,4596.5916,4.106700,5.972900,7.426800,4.028100,700.0,...,False,False,False,False,False,False,False,False,False,False
2,133.398074,0.0,0.0,24969.9701,4478.5564,3.195182,5.538617,7.017926,3.167118,700.0,...,False,False,False,False,False,False,False,False,False,False
3,135.133852,0.0,0.0,27082.9626,4323.3712,2.491792,5.249173,6.700315,2.563862,700.0,...,False,False,False,False,False,False,False,False,False,False
4,131.699424,0.0,0.0,26890.9717,4231.8283,1.996528,5.104570,6.473967,2.218332,700.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,171.707318,0.0,0.0,36997.7200,1108.4000,8.530000,9.960000,10.790000,9.590000,1200.0,...,False,False,False,False,False,True,False,False,False,False
26300,159.462903,0.0,0.0,35666.9300,1077.9700,8.000000,9.400000,10.630000,9.110000,1200.0,...,False,False,False,False,False,False,True,False,False,False
26301,155.109520,0.0,0.0,34383.8800,1048.2800,7.500000,8.880000,10.510000,8.670000,1200.0,...,False,False,False,False,False,False,False,True,False,False
26302,171.370277,0.0,0.0,33075.2500,1078.7800,6.970000,8.510000,10.320000,8.140000,1200.0,...,False,False,False,False,False,False,False,False,True,False


In [116]:
df_encoded.isna().sum()

MWh                    0
solar_fore_de [MW]     0
solar_fore_it [MW]     0
wind_fore_de [MW]      0
wind_fore_it [MW]      0
temperature_fore_ch    0
temperature_fore_fr    0
temperature_fore_de    0
temperature_fore_it    0
CH_AT                  0
CH_DE                  0
CH_FR                  0
CH_IT                  0
AT_CH                  0
DE_CH                  0
FR_CH                  0
IT_CH                  0
month_1                0
month_2                0
month_3                0
month_4                0
month_5                0
month_6                0
month_7                0
month_8                0
month_9                0
month_10               0
month_11               0
month_12               0
weekday_0              0
weekday_1              0
weekday_2              0
weekday_3              0
weekday_4              0
weekday_5              0
weekday_6              0
time_00:00:00          0
time_01:00:00          0
time_02:00:00          0
time_03:00:00          0


# Model


In [117]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils import clip_grad_norm_

In [118]:
# Extract features and labels
labels = df_encoded['MWh'].values
features = df_encoded.drop(['MWh'], axis=1).values

In [119]:
# Normalize features using Min-Max scaling
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [120]:
# Define sequence length (number of time steps for LSTM)
sequence_length = 7*24  # Use the previous 7 days as input, 24 measurements

In [121]:
# Create sequences and labels for training
X, y = [], []
for i in range(len(features) - sequence_length):
    X.append(features[i:i+sequence_length])
    y.append(labels[i+sequence_length])

X, y = np.array(X), np.array(y)

In [122]:
# Split data into training and testing sets
split_ratio = 0.8  # Adjust as needed
split_index = int(split_ratio * len(X))


X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [123]:

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [124]:
# Create DataLoader for training and testing
batch_size = 32  # Adjust as needed
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [125]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the output from the last time step
        return out

In [126]:
class BidirectionalLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.2):
        super(BidirectionalLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Double hidden_size due to bidirectional LSTM

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the output from the last time step
        return out

In [127]:
input_size = X_train.shape[2]  # Number of features
hidden_size = 64  # Adjust as needed
num_layers = 14  # Adjust as needed
output_size = 1  # Adjust for regression tasks

In [128]:


model = BidirectionalLSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.L1Loss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate as needed

scheduler = StepLR(optimizer, step_size=5, gamma=0.5)




In [129]:
# Define the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Move the model to the GPU
model.to(device)

# Move data to GPU
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [ ]:
model.to(device)
model.train()
num_epochs = 50  # Adjust as needed
# Define the maximum gradient norm threshold
max_grad_norm = 1.0  # Adjust as needed

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the same device as the model
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        
        # Clip gradients to prevent exploding gradients
        clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    scheduler.step()



Epoch [1/50], Loss: 24.3428
Epoch [2/50], Loss: 24.0247
Epoch [3/50], Loss: 24.4381
Epoch [4/50], Loss: 24.2555
Epoch [5/50], Loss: 24.3261
Epoch [6/50], Loss: 40.2525
Epoch [7/50], Loss: 40.2797
Epoch [8/50], Loss: 40.3034
Epoch [9/50], Loss: 40.3252
Epoch [10/50], Loss: 40.3384
Epoch [11/50], Loss: 43.5339
Epoch [12/50], Loss: 43.6188
Epoch [13/50], Loss: 43.6004
Epoch [14/50], Loss: 43.5823
Epoch [15/50], Loss: 43.5643
Epoch [16/50], Loss: 43.4839
Epoch [17/50], Loss: 43.5369
Epoch [18/50], Loss: 43.5453
Epoch [19/50], Loss: 43.5536
Epoch [20/50], Loss: 43.5620
Epoch [21/50], Loss: 43.0376
Epoch [22/50], Loss: 42.8989
Epoch [23/50], Loss: 42.8433
Epoch [24/50], Loss: 42.8299
Epoch [25/50], Loss: 42.8573
Epoch [26/50], Loss: 42.7745
Epoch [27/50], Loss: 42.7420
Epoch [28/50], Loss: 42.7095
Epoch [29/50], Loss: 42.7104
Epoch [30/50], Loss: 42.7114
Epoch [31/50], Loss: 42.7140
Epoch [32/50], Loss: 42.7311
Epoch [33/50], Loss: 42.7482
Epoch [34/50], Loss: 42.7550


In [ ]:

# Rolling window evaluation
model.eval()
predictions = []

for i in range(len(X_test)):
    if i < sequence_length:
        # Initial predictions using the training data
        predictions.append(y_train[i].item())
    else:
        inputs = X_test[i].unsqueeze(0)
        with torch.no_grad():
            output = model(inputs)
        predictions.append(output.item())


        

In [ ]:
# Calculate and print evaluation metrics (e.g., MAE, MSE, etc.)
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Move the predictions tensor from GPU to CPU
predictions = np.array(predictions)
y_test_cpu = y_test.cpu().numpy()

mae = mean_absolute_error(y_test_cpu, predictions)
mse = mean_squared_error(y_test_cpu, predictions)
print(f'MAE: {mae:.4f}')
print(f'MSE: {mse:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))


# Convert predictions to a NumPy array
predictions = predictions
y_test = y_test.cpu().numpy()

# Generate a numerical index from 0 to len(y_train)-1
index = np.arange(len(y_test))

# Plot the true values against the numerical index
ax.plot(index, y_test, label='True Values', marker='o', linestyle='-')

# Plot the predicted values against the numerical index
ax.plot(index, predictions, label='Predicted Values', marker='x', linestyle='-')

# Add labels and legend
ax.set_title('True vs. Predicted Values')
ax.set_xlabel('Index')
ax.set_ylabel('Values')
ax.legend()

# Display the plot
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
len(y_test)